In [1]:
!pip install catboost, xgboost

In [2]:
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# Đọc dữ liệu từ file data.pkl.File nay chua du lieu da duoc can bang voi WGAN va ENN-Kmean
with open('cse-cic-ids2018-balanced.pkl', 'rb') as file:
    data_loaded = pickle.load(file)

X_train = data_loaded['X_train']
X_test = data_loaded['X_test']
y_train = data_loaded['y_train']
y_test = data_loaded['y_test']


In [ ]:
unique_values, counts = np.unique(y_train, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

Brute Force -Web: 7000
Brute Force -XSS: 7000
DDOS attack-LOIC-UDP: 7000
DoS attacks-GoldenEye: 7000
DoS attacks-Slowloris: 7000


In [ ]:
unique_values, counts = np.unique(y_test, return_counts=True)

# Hiển thị các giá trị và tần suất xuất hiện tương ứng
for value, count in zip(unique_values, counts):
    print(f'{value}: {count}')

Brute Force -Web: 183
Brute Force -XSS: 69
DDOS attack-LOIC-UDP: 519
DoS attacks-GoldenEye: 3000
DoS attacks-Slowloris: 3000


In [ ]:
X_train[:10]

array([[1.22074038e-03, 3.52941176e-01, 1.92369443e-01, 1.10864745e-04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.74927499e-08, 1.92383036e-01, 0.00000000e+00, 1.92383032e-01,
        1.92383039e-01, 1.92369612e-01, 1.92383039e-01, 0.00000000e+00,
        1.92383039e-01, 1.92383039e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.12317002e-04,
        0.00000000e+00, 1.87310667e-08, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [ ]:
y_train[:5]

array(['DoS attacks-GoldenEye', 'DoS attacks-GoldenEye',
       'DoS attacks-GoldenEye', 'DoS attacks-GoldenEye',
       'DoS attacks-GoldenEye'], dtype='<U21')

In [ ]:
# Assuming X_train, X_test, y_train, y_test are already defined and preprocessed.
num_classes = len(np.unique(np.concatenate([y_train, y_test])))

# Define and train models
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train, y_train)

cbt = CatBoostClassifier(verbose=0)
cbt.fit(X_train, y_train)

gbm = GradientBoostingClassifier()
gbm.fit(X_train, y_train)

bme = BaggingClassifier(base_estimator=RandomForestClassifier(), n_estimators=10)
bme.fit(X_train, y_train)

# Deep Neural Network
dnn = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])
dnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
dnn.fit(X_train, to_categorical(y_train, num_classes=num_classes), epochs=10, batch_size=32, verbose=0)

# Predict probabilities
prob_xgb = xgb.predict_proba(X_test)
prob_cbt = cbt.predict_proba(X_test)
prob_gbm = gbm.predict_proba(X_test)
prob_bme = bme.predict_proba(X_test)
prob_dnn = dnn.predict(X_test)  # This already gives probabilities due to softmax



In [ ]:
def evaluate_model(y_true, predictions):
    acc = accuracy_score(y_true, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, predictions, average='weighted')
    return acc, precision, recall, f1

In [ ]:
# Convert predicted probabilities to class labels for evaluation
predictions_xgb = np.argmax(prob_xgb, axis=1)
predictions_cbt = np.argmax(prob_cbt, axis=1)
predictions_gbm = np.argmax(prob_gbm, axis=1)
predictions_bme = np.argmax(prob_bme, axis=1)
predictions_dnn = np.argmax(prob_dnn, axis=1)

# Evaluate each model
results_xgb = evaluate_model(y_test, predictions_xgb)
results_cbt = evaluate_model(y_test, predictions_cbt)
results_gbm = evaluate_model(y_test, predictions_gbm)
results_bme = evaluate_model(y_test, predictions_bme)
results_dnn = evaluate_model(y_test, predictions_dnn)

# Print results for each model
print("XGB - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_xgb))
print("CBT - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_cbt))
print("GBM - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_gbm))
print("BME - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_bme))
print("DNN - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(*results_dnn))


In [ ]:
# Ensemble learning

# Weighted average of probabilities
ensemble_probabilities = np.average(
    [prob_xgb, prob_cbt, prob_gbm, prob_bme, prob_dnn],
    axis=0,
    weights=[0.3, 0.2, 0.2, 0.2, 0.1]
)

# Final prediction is the class with the highest average probability
ensemble_predictions = np.argmax(ensemble_probabilities, axis=1)

# Evaluate ensemble
ensemble_results = evaluate_model(y_test, ensemble_predictions)

print("Ensemble Accuracy:", ensemble_results[0])
print("Ensemble Precision:", ensemble_results[1])
print("Ensemble Recall:", ensemble_results[2])
print("Ensemble F1-Score:", ensemble_results[3])